#### General Code Setup
As a first step we:
- Import all the necessary libraries for our computation
- Setup some folders for the loading and saving of all necessary files and to run correctly EMUstack

#### Libraries imported
- [os](https://docs.python.org/3/library/os.html): used to setup some folder to run correctly the code
- [sys](https://docs.python.org/3/library/sys.html): used to make EMUstack accessible
- [numpy](https://numpy.org/): essentially matlab for python. We use it to manipulate matrix data
- [concurrent.futures](https://docs.python.org/3/library/concurrent.futures.html): to run the code in parallel
- [xarray](https://docs.xarray.dev/en/stable/): library to save labelled data along with the relevant metadata (important for reproducibility)
- [datetime](https://docs.python.org/3/library/datetime.html): we use it to create a timestamp for the saved files

In [35]:
# libraries
import os
import numpy as np
import sys
import concurrent.futures
import xarray as xr
from datetime import datetime

# folder setup
home_dir = os.environ["HOME"]  # home folder
script_dir = os.getcwd()  # running file folder
s4_dir = [s for s in os.listdir(home_dir + '/programs/S4/build/') if "lib.linux" in s][0]  # S4 library subfolder
utils_dir = '../'  # folder for utilities
materials_dir = '../materials/'  # folder for material files

#### Import S4
In order to correctly load all the components of S4 we:
- make the S4 folder accessible
- load the S4 module
- load some utilities to handle the optical constants, because S4 does not have a builting feature for this

In [36]:
# importing S4
sys.path.append(home_dir + '/programs/S4/build/' + s4_dir)
sys.path.append(utils_dir)
import S4
import utils

In [37]:
# building the optical constant database
eps_db_out=utils.generate_eps_db(materials_dir,ext='*.edb')  # generates a dictionary with files, eps names and a dictionary containg everything
eps_files,eps_names,eps_db=eps_db_out['eps_files'],eps_db_out['eps_names'],eps_db_out['eps_db']  # separate the 3 outputs

#### Defining the inputs
We are going to calculate the transmission and reflectance spectrum of a Gold Nanodisk array. To do so we need to define the following groups of input parameters:
- **Geometrical parameters** of our multilayers, including thicknesses, disk size, etc...
- **Material parameters** such as the composition of each layer, substrate and superstrate included
- **Illumination parameters**, i.e. wavelengths, angles and polarizations of the incident field
- **S4 parameters**, i.e. parameters that are strictly connected with the inner workings of S4

We will try to store all these input parameters inside [dictionaries](https://docs.python.org/3/tutorial/datastructures.html#dictionaries), so that they can be handily exported later in order to improve the riproducibility of our calculations

In [38]:
# light dictionary
light = {}
light['theta'] = 0.0
light['phi'] = 0.0
light['pol'] = [0.0,1.0]  # s and p polarization amplitudes
light['NumBasis'] = 100  # number of plane waves

# Geometrical parameters
struct = {}
# nanodisk array parameters
struct['inc_shape'] = 'circle'  # flag to log the shape of the inclusion
struct['nd_radius'] = 100.0  # nanodisk radius
struct['nd_height'] = 100.0  # nanodisk height
struct['nd_period_x'] = 600.0  # nanodisk array x period
struct['nd_period_y'] = 600.0  # nanodisk array y period
struct['nd_inc_mat'] = 'e_au'  # nanodisk material
struct['nd_back_mat'] = 'e_vacuum'  # background material in the nanodisk array
# superstrate and substrate parameters
struct['sub_mat'] = 'e_vacuum'  # substrate material
struct['sup_mat'] = 'e_vacuum'  # superstrate material
struct['materials_stack'] = [struct["sup_mat"], struct["nd_inc_mat"], struct["sub_mat"]]

# Emustack parameters
s4 = {}
s4['PolarizationDecomposition'] = True  # are we using fast fourier factorization
s4['PolarizationBasis'] = 'Jones'  # which kind of fast fourier factorization?

#### Load spectral data
We load the spectra data. We then take the wavelength of maximum reflectance to compute the local fields

In [39]:
s4_spectra = xr.load_dataarray('data/S4_AuNdArraySpectra_circle_px600.0_py400.0_r100.0_h100.0_180222_123031.nc')  # load the spectral data
wl_index = s4_spectra.sel(spectrum='R').argmax().item()  # find the index where the reflectance is maximum
light['wl'] = s4_spectra.coords['wavelength'][wl_index].item()  # get the wavelength of maximum reflactance

#### Build the S4 stack
In S4 you initialize a whole **S** object, which containts all the informations necessary for the computation, including:
- Incident plane wave, energy, direction, polarization, and number of **G** orders included
- Formal definition of the materials
- Information on all the layers, nanostructured or not, from top to bottom
- Computational informations, such as fast fourier factorization, etc...

In [40]:
# initialize s4 lattice, including the number of plane waves
S = S4.New(Lattice=((struct["nd_period_x"], 0), (0, struct["nd_period_y"])), NumBasis=light["NumBasis"])

# retrieving optical constants at wl from the database
e_list = np.array(utils.db_to_eps(light['wl'], eps_db, struct['materials_stack']))

# Define the materias inside the S4 formalism: the materials are complex dielectric functions
S.SetMaterial("Air", e_list[0])
S.SetMaterial("Au", e_list[1])

# Add the incident medium layer
S.AddLayer("Inc", 0.0, "Air")  # Add layer with a Name, a Thickness and a Composition

# Add the Nanostructured layer
S.AddLayer("Slab", struct["nd_height"], "Air")
S.SetRegionCircle("Slab", "Au", (struct['nd_period_x']/2, struct['nd_period_y']/2), struct["nd_radius"])

# Substrate
S.AddLayer("Sub", 0.0, "Air")

# incident wave + computing options
S.SetExcitationPlanewave((light["theta"], light["phi"]), light["pol"][0], light["pol"][1])
S.SetFrequency(1.0/light['wl'])  # frequency as reciprocal of the utilized wavelength)
S.SetOptions(PolarizationDecomposition=s4["PolarizationDecomposition"], PolarizationBasis=s4["PolarizationBasis"])



#### LF computation
Now we build functions that:
- takes and incident field, i.e. and object.Light as input
- solves for the mode in each layer, where the FEM solver works in the nanostructured layer
- builds a stack with the solver layer, in the order bottom -> top
- returns the stack
- take the returned stack as input and compute the local field input

In [41]:
def lf(S, z=0.0, n_points=500):

    # Conpute fields on a grid using s4 native routine
    E,H = S.GetFieldsOnGrid(z = z, NumSamples=(n_points,n_points), Format = 'Array')

    # compute square modulus of the field
    E = np.array(list(E))
    m_AbsE = np.sqrt(np.abs(E[:,:,0])**2 + np.abs(E[:,:,1])**2 + np.abs(E[:,:,2])**2).T

    return m_AbsE

In [42]:
m_AbsE = lf(S,z=0.5 * struct['nd_height'])

In [43]:
# build output filename
now = datetime.now()  # what time is it?
timestamp = now.strftime("%d%m%y_%H%M%S")  # build timestamp string as day,month,year _ hour,minute,second
out_filename = (
    "S4_AuNdArrayLF_"
    + struct["inc_shape"]
    + "_px"
    + str(struct["nd_period_x"])
    + "_py"
    + str(struct["nd_period_y"])
    + "_r"
    + str(struct["nd_radius"])
    + "_h"
    + str(struct["nd_height"])
    + "_"
    + timestamp
)

# create vector to map x and y coords
v_coords_x = np.linspace(0.0,struct['nd_period_x'],500)
v_coords_y = np.linspace(0.0,struct['nd_period_y'],500)
# save the data as in the case of the spectra
s4_lf = xr.DataArray(m_AbsE, coords=[v_coords_y,v_coords_x], dims=["y", "x"])
s4_lf.attrs['light'] = str(light)
s4_lf.attrs['struct'] = str(struct)
s4_lf.attrs['emu'] = str(s4)
s4_lf.to_netcdf("data/" + out_filename + ".nc")